In [69]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import json

# 配置参数
CONFIG = {
    'model_path': '/home/eton/job-trainThyUS/59-result-models/model-efficientnet-eton251024/nodule_feature_cnn_v75/nodule_feature_cnn_v75_best_auc.pth',
    'feature_mapping_file': '/home/eton/job-trainThyUS/71-datasets/251016-efficientNetDatas/dataset/all_features_mapping_numer_v4.json',
    'threshold': 0.5,
    'imageNet_normalized_mean': [0.485, 0.456, 0.406],
    'imageNet_normalized_std': [0.229, 0.224, 0.225]
}

# 获取设备
def get_device():
    if torch.backends.mps.is_available(): return torch.device("mps")
    if torch.cuda.is_available(): return torch.device("cuda")
    return torch.device("cpu")

# 定义模型（需要与训练时完全相同）
class MultiTaskNoduleCNN(nn.Module):
    def __init__(self, feature_mappings, dropout_rate=0.4):
        super().__init__()
        self.mappings = feature_mappings
        
        from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
        self.backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        backbone_features = 1280
        self.backbone.classifier = nn.Identity()
        
        self.shared_features = nn.Sequential(
            nn.Linear(backbone_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.75)
        )
        
        self.heads = nn.ModuleDict()
        for task, mapping in self.mappings.items():
            num_classes = len(mapping)
            print(" -num_classes=",num_classes, task, mapping)
            self.heads[task] = nn.Sequential(
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(128, num_classes)
            )
            
    def forward(self, x):
        features = self.backbone(x)
        shared = self.shared_features(features)
        outputs = {}
        for task, head in self.heads.items():
            outputs[task] = head(shared)
        return outputs

# 单张图像预测函数
def predict_single_image(model, image_path, transform, device, threshold=0.5):
    # 打开并预处理图像
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 添加批次维度
    image = image.to(device)
    
    # 推理
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        print("outputs:", outputs)
        bom_logits = outputs['bom']
        print("bom-logits:", type(bom_logits), type(bom_logits.data), bom_logits.data)
        max_values, max_indices = torch.max(bom_logits, dim=1)
        print("bom-logits:", max_values, max_indices)
        bom_softmax = torch.softmax(bom_logits, dim=1)
        bom_probs = bom_softmax[:, 1].item()
        print("bom_logits softmax:", bom_softmax, bom_softmax[:, 1], bom_probs)
        benignProbs = bom_softmax[0, 0].item()  # Probability of class 0
        MalignProbs = bom_softmax[0, 1].item()  # Probability of class 1
        prediction = 1 if MalignProbs > benignProbs else 0
    
    return {
        'prediction': prediction,
        'confidence': bom_probs,
        'label': '恶性' if prediction == 1 else '良性'
    }

# 主函数
def main():
    # 获取设备
    device = get_device()
    print(f"使用设备: {device}")
    
    # 检查模型文件
    if not os.path.exists(CONFIG['model_path']):
        print(f"错误: 模型文件不存在: {CONFIG['model_path']}")
        return
    
    # 加载特征映射
    if not os.path.exists(CONFIG['feature_mapping_file']):
        print(f"错误: 特征映射文件不存在: {CONFIG['feature_mapping_file']}")
        return
    
    with open(CONFIG['feature_mapping_file'], 'r', encoding='utf-8') as f:
        feature_mapping = json.load(f)
        print("feature_mapping:", feature_mapping)
    
    # 创建模型并加载权重
    model = MultiTaskNoduleCNN(feature_mapping, dropout_rate=0.4).to(device)
    model.load_state_dict(torch.load(CONFIG['model_path'], map_location=device))
    print("模型加载成功")
    
    # 定义图像预处理转换
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 示例：预测单张图像
    # image_path = input("请输入要预测的图像路径: ")
    image_path=r"/tmp/input01/02.202408010081.01/02.202408010081.01.20177.0003.08140600191.jpg"
    if os.path.exists(image_path):
        result = predict_single_image(model, image_path, val_transform, device, CONFIG['threshold'])
        print(f"预测结果: {result['label']}")
        print(f"恶性概率: {result['confidence']:.4f}")
    else:
        print(f"错误: 图像文件不存在: {image_path}")



In [70]:
main()

使用设备: cpu
feature_mapping: {'bom': {'0': 0, '1': 1}, 'ti_rads': {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5}, 'composition': {'囊性': 0, '囊实性': 1, '实性': 2}, 'echo': {'海绵样': 0, '无回声': 1, '高回声': 2, '等回声': 3, '强回声': 4, '不均质回声': 5, '低回声': 6, '极低回声': 7}, 'foci': {'无点状强回声': 0, '彗星尾征': 1, '无回声区': 2, '高回声区': 3, '粗大钙化': 4, '斑状强回声': 5, '弧形强回声/环状强回声': 6, '点状强回声': 7}, 'margin': {'清楚': 0, '尚清': 1, '欠清': 2, '不清': 3}, 'shape': {'规则': 0, '尚规则': 1, '欠规则': 2, '不规则': 3}}
 -num_classes= 2 bom {'0': 0, '1': 1}
 -num_classes= 5 ti_rads {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5}
 -num_classes= 3 composition {'囊性': 0, '囊实性': 1, '实性': 2}
 -num_classes= 8 echo {'海绵样': 0, '无回声': 1, '高回声': 2, '等回声': 3, '强回声': 4, '不均质回声': 5, '低回声': 6, '极低回声': 7}
 -num_classes= 8 foci {'无点状强回声': 0, '彗星尾征': 1, '无回声区': 2, '高回声区': 3, '粗大钙化': 4, '斑状强回声': 5, '弧形强回声/环状强回声': 6, '点状强回声': 7}
 -num_classes= 4 margin {'清楚': 0, '尚清': 1, '欠清': 2, '不清': 3}
 -num_classes= 4 shape {'规则': 0, '尚规则': 1, '欠规则': 2, '不规则': 3}
模型加载成功
outputs: {'bom': tensor([[-0

In [20]:
device = get_device()
with open(CONFIG['feature_mapping_file'], 'r', encoding='utf-8') as f:
    feature_mapping = json.load(f)
model = MultiTaskNoduleCNN(feature_mapping, dropout_rate=0.4).to(device)

OrderedDict=torch.load(CONFIG['model_path'], map_location=device)

In [21]:
model.load_state_dict(OrderedDict)

<All keys matched successfully>

In [22]:
help(weights)

Help on OrderedDict object:

class OrderedDict(builtins.dict)
 |  Dictionary that remembers insertion order
 |  
 |  Method resolution order:
 |      OrderedDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __or__(self, value, /)
 |      Return self|value.
 |  
 |  __reduce__(...)
 |      Return state info

In [ ]:
for i in OrderedDict.items():
    print(i)

In [24]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

In [25]:
dir(backbone)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_impl',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_per

In [26]:
help(backbone.features)

Help on Sequential in module torch.nn.modules.container object:

class Sequential(torch.nn.modules.module.Module)
 |  Sequential(*args)
 |  
 |  A sequential container.
 |  
 |  Modules will be added to it in the order they are passed in the
 |  constructor. Alternatively, an ``OrderedDict`` of modules can be
 |  passed in. The ``forward()`` method of ``Sequential`` accepts any
 |  input and forwards it to the first module it contains. It then
 |  "chains" outputs to inputs sequentially for each subsequent module,
 |  finally returning the output of the last module.
 |  
 |  The value a ``Sequential`` provides over manually calling a sequence
 |  of modules is that it allows treating the whole container as a
 |  single module, such that performing a transformation on the
 |  ``Sequential`` applies to each of the modules it stores (which are
 |  each a registered submodule of the ``Sequential``).
 |  
 |  What's the difference between a ``Sequential`` and a
 |  :class:`torch.nn.ModuleLi

In [33]:
# 通过检查模型的第一层来获取输入大小
def get_input_size_from_model(model):
    if model is None:
        return None
    
    # 由于模型使用了 EfficientNet-B0，我们知道它期望 224x224 大小的输入
    # 但如果需要动态获取，可以尝试以下方法
    try:
        # 尝试获取第一层卷积的输入通道数
        first_conv = None
        # 遍历模型的 backbone 查找第一层卷积
        for module in model.modules():
            print(module)
            if isinstance(module, nn.Conv2d):
                first_conv = module
                break
        
        if first_conv:
            # 返回 (高度, 宽度, 通道数)
            # 注意：高度和宽度需要参考预处理设置
            print("first conv:", first_conv, "first_conv.in_channels=", first_conv.in_channels)
            return (224, 224, first_conv.in_channels)
        else:
            return (224, 224, 3)  # 默认 RGB 图像
    except Exception as e:
        print(f"Error getting input size from model: {str(e)}")
        return (224, 224, 3)  # 返回默认值

In [34]:
get_input_size_from_model(backbone)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

(224, 224, 3)

In [37]:
print(backbone._parameters)

{}
